# 입문자를 위한, 파이썬/R 데이터 분석      

]

##  (공공데이터분석) 국토부 아파트매매 실거래가 API[data.go.kr] 연동

#### XML 국토교통부_아파트매매 실거래자료 : https://www.data.go.kr/data/15058747/openapi.do

#### 행정표준관리시스템 : https://www.code.go.kr/index.do ( [메뉴] 자주이용하는 코드 > 법정동 > [법정동코드전체자료] )

]

### 1. Import Packages

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### 2. Data Loading 

#### : API Key Setting

In [2]:
f=open("data/my_api_key.txt","r")
lines=f.readlines()
my_key=lines[0]
f.close()

In [3]:
my_key

'XKlHbJN1QExVlpzVxzP9X%2BF3oqBJwH%2FGOrP2iayTTwfTzvSHIQJJijgYMQ8sk9IqG%2F9CKNQO1o2tDCL5zLA2jw%3D%3D'

#### : Pre-defined

In [4]:
#"http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade
#        ?
#        LAWD_CD=11110
#        &
#        DEAL_YMD=201512
#        &
#        serviceKey=서비스키"

In [5]:
base_url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade"

In [6]:
area_cd = "LAWD_CD=" + "11680"    # 11680 : 강남구
deal_dt = "DEAL_YMD=" + "202404"
svc_key = "serviceKey=" + my_key

In [7]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11680&DEAL_YMD=202404&serviceKey=XKlHbJN1QExVlpzVxzP9X%2BF3oqBJwH%2FGOrP2iayTTwfTzvSHIQJJijgYMQ8sk9IqG%2F9CKNQO1o2tDCL5zLA2jw%3D%3D'

#### : Request Test

In [8]:
response = requests.get(url)
response

<Response [200]>

In [9]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><거래금액>     245,000</거래금액><거래유형>중개거래</거래유형><건축년도>2006</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>역삼푸르지오</아파트><월>4</월><일>2</일><전용면적>84.9097</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>754-1</지번><지역코드>11680</지역코드><층>16</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>     268,000</거래금액><거래유형>중개거래</거래유형><건축년도>2012</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>개나리SK뷰</아파트><월>4</월><일>5</일><전용면적>84.99</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>716-3</지번><지역코드>11680</지역코드><층>4</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>     196,000</거래금액><거래유형>중개거래</거래유형><건축년도>2005</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>e-편한세상</아파트><월>4</월><일>5</일><전용면적>59.606</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>755-4</지번><지역코드>11680</지역코드><층

### Addtional Preparation : 

#### -1)  Input Data : lawd_cd / 법정동 코드(지역코드) 가공

In [10]:
lawd_cd = pd.read_csv("data/법정동코드 전체자료.txt", sep='\t', encoding ='cp949') #'euc-kr', 'utf-8'
lawd_cd

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
49845,5280042024,전북특별자치도 부안군 위도면 대리,존재
49846,5280042025,전북특별자치도 부안군 위도면 거륜리,존재
49847,5280042026,전북특별자치도 부안군 위도면 식도리,존재
49848,5280042027,전북특별자치도 부안군 위도면 상왕등리,존재


In [11]:
# Column 이름 변경 : 데이터 조작 편의성을 위함
print(lawd_cd.columns)

Index(['법정동코드', '법정동명', '폐지여부'], dtype='object')


In [12]:
lawd_cd.columns = ["code", "name", "check"]
lawd_cd

,code,name,check
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
49845,5280042024,전북특별자치도 부안군 위도면 대리,존재
49846,5280042025,전북특별자치도 부안군 위도면 거륜리,존재
49847,5280042026,전북특별자치도 부안군 위도면 식도리,존재
49848,5280042027,전북특별자치도 부안군 위도면 상왕등리,존재


In [13]:
lawd_cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49850 entries, 0 to 49849
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    49850 non-null  int64 
 1   name    49850 non-null  object
 2   check   49850 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [14]:
in_name = "종로구"

In [15]:
lawd_cd['name'].str.contains(in_name)

0        False
1         True
2         True
3         True
4         True
         ...  
49845    False
49846    False
49847    False
49848    False
49849    False
Name: name, Length: 49850, dtype: bool

In [16]:
lawd_cd['check'] == "존재"

0        True
1        True
2        True
3        True
4        True
         ... 
49845    True
49846    True
49847    True
49848    True
49849    True
Name: check, Length: 49850, dtype: bool

In [17]:
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")]

,code,name,check
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
5,1111010400,서울특별시 종로구 효자동,존재
...,...,...,...
84,1111018300,서울특별시 종로구 평창동,존재
85,1111018400,서울특별시 종로구 부암동,존재
86,1111018500,서울특별시 종로구 홍지동,존재
87,1111018600,서울특별시 종로구 신영동,존재


In [18]:
tmp = lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1)
tmp

,code,name,check
1,1111000000,서울특별시 종로구,존재


In [19]:
tmp.loc[1,'code']

1111000000

In [20]:
tmp.iloc[0,0]

1111000000

In [21]:
tmp = tmp.iloc[0,0]
#str(tmp)
tmp = tmp.astype(str)
tmp[0:5]

'11110'

In [22]:
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]

'11110'

In [23]:
# Function 작성 :
def find_lawd_cd(in_text):
    return lawd_cd[(lawd_cd['name'].str.contains(in_text)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]    

In [24]:
in_name = "구로구"
find_lawd_cd(in_name)

'11530'

#### -2)  Input Data : lawd_cd, deal_ymd 입력받기

In [44]:
in_lawd = input("검색지역 입력 : ")

검색지역 입력 : 강남구


In [45]:
in_lawd

'강남구'

In [46]:
in_deal_ymd = input("계약월 입력(예: 201912) : ")

계약월 입력(예: 201912) : 202402


In [47]:
in_deal_ymd

'202402'

In [48]:
in_svckey = my_key

In [49]:
in_lawd_cd = find_lawd_cd(in_lawd)
in_lawd_cd

'11680'

In [50]:
area_cd = "LAWD_CD=" + in_lawd_cd
deal_dt = "DEAL_YMD=" + in_deal_ymd
svc_key = "serviceKey=" + in_svckey

In [51]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11680&DEAL_YMD=202402&serviceKey=XKlHbJN1QExVlpzVxzP9X%2BF3oqBJwH%2FGOrP2iayTTwfTzvSHIQJJijgYMQ8sk9IqG%2F9CKNQO1o2tDCL5zLA2jw%3D%3D'

In [52]:
response = requests.get(url)
response

<Response [200]>

In [53]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><거래금액>     233,000</거래금액><거래유형>중개거래</거래유형><건축년도>2006</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>역삼푸르지오</아파트><월>2</월><일>2</일><전용면적>84.9097</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>754-1</지번><지역코드>11680</지역코드><층>7</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>     197,500</거래금액><거래유형>중개거래</거래유형><건축년도>2005</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>e-편한세상</아파트><월>2</월><일>2</일><전용면적>59.606</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>755-4</지번><지역코드>11680</지역코드><층>25</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>      69,000</거래금액><거래유형>중개거래</거래유형><건축년도>2006</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>역삼I\'PARK</아파트><월>2</월><일>3</일><전용면적>28.246</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>713-11</지번><지역코드>11680</지역

### 3. Data Preprocessng 

#### : web data / API output

In [54]:
soup = BeautifulSoup(response.text, "lxml")
soup

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>00</resultcode><resultmsg>NORMAL SERVICE.</resultmsg></header><items><item>&lt;거래금액&gt;     233,000거래금액&gt;&lt;거래유형&gt;중개거래거래유형&gt;&lt;건축년도&gt;2006건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt;개인매도자&gt;&lt;매수자&gt;개인매수자&gt;&lt;법정동&gt; 역삼동법정동&gt;&lt;아파트&gt;역삼푸르지오아파트&gt;&lt;월&gt;2월&gt;&lt;일&gt;2일&gt;&lt;전용면적&gt;84.9097전용면적&gt;&lt;중개사소재지&gt;서울 강남구중개사소재지&gt;&lt;지번&gt;754-1지번&gt;&lt;지역코드&gt;11680지역코드&gt;&lt;층&gt;7층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;해제여부&gt; 해제여부&gt;</item><item>&lt;거래금액&gt;     197,500거래금액&gt;&lt;거래유형&gt;중개거래거래유형&gt;&lt;건축년도&gt;2005건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt;개인매도자&gt;&lt;매수자&gt;개인매수자&gt;&lt;법정동&gt; 역삼동법정동&gt;&lt;아파트&gt;e-편한세상아파트&gt;&lt;월&gt;2월&gt;&lt;일&gt;2일&gt;&lt;전용면적&gt;59.606전용면적&gt;&lt;중개사소재지&gt;서울 강남구중개사소재지&gt;&lt;지번&gt;755-4지번&gt;&lt;지역코드&gt;11680지역코드&gt;&lt;층&gt;25층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;

In [36]:
soup.find_all('item')

[]

In [37]:
item = soup.find('item')

In [38]:
#item.find('거래금액').text

In [39]:
import xml.etree.ElementTree as ET

In [40]:
response#.content#.text

<Response [200]>

In [41]:
root = ET.fromstring(response.content)
root

<Element 'response' at 0x0000014B61FBFC20>

In [42]:
item_list = []

In [43]:
root.find('body').find('items')[0]

IndexError: child index out of range

In [ ]:
type(root.find('body').find('items'))

In [ ]:
child = root.find('body').find('items')[0]
child.findall('*')#[0]

In [ ]:
element = child.findall('*')[0]

In [ ]:
element.tag.strip()

In [ ]:
element.text.strip()

In [ ]:
data = {}

In [ ]:
tag = element.tag.strip()
text = element.text.strip()
data[tag] = text

In [ ]:
data

In [ ]:
def find_xml_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list

In [ ]:
items_list = find_xml_items(response)
items_list

### 4. Result : Display & Save as file 

In [ ]:
result = pd.DataFrame(items_list)

In [ ]:
result

In [ ]:
result.shape

In [ ]:
result.head()

#### : file name

In [ ]:
name_info = in_lawd + "_" + in_deal_ymd
name_info

In [ ]:
file_name = f"data/result_{name_info}.csv"
file_name

In [ ]:
result.to_csv(file_name, index=False)

## Appendix : Function 

In [ ]:
def deal_by_lawd(cd, ymd):
    
    svckey = my_key
    
    area_cd = "LAWD_CD=" + cd
    deal_dt = "DEAL_YMD=" + ymd
    svc_key = "serviceKey=" + svckey
    url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key

    response = requests.get(url)
    items_list = find_xml_items(response)
    result = pd.DataFrame(items_list)
    
    name_info = in_lawd + "_" + in_deal_ymd
    file_name = f"data/result_{name_info}.csv"

    result.to_csv(file_name, index=False)
    
    return file_name

In [ ]:
#### - Input Data : LAWD_CD, DEAL_YMD, serviceKey  입력받기
in_lawd = input("검색지역 입력 : ")
print(in_lawd)

in_lawd_cd = find_lawd_cd(in_lawd)
print(in_lawd_cd)

in_deal_ymd = input("계약월 입력(예: 201911) : ")
print(in_deal_ymd)

In [ ]:
deal_by_lawd(in_lawd_cd, in_deal_ymd)

### 추가 실습 : 
- 공공 API 데이터 끌어오기 친숙해지기
- 특정 월에 서울시 전체 구의 데이터를 한 번에 가져오기(입력값 : 년월 20402)
- 서울시의 특정 하나의 구의 과거 1년치 데이터를 한번에 가져오기(입력값 : 특정구 하나)